## 1) Librerias

In [1]:
import os
import time
import joblib
import cv2 as  cv
import numpy as np
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

## 2) Lectura de imagenes

### Métodos

In [5]:
### COLOCARSE DENTRO DE LA CARPETA 'Data'
#os.chdir("Data_CarsType") #Data de Tipo de Carros
os.chdir("Data_CatDogBird") #Data de Gatos-Perros-Pájaro
os.listdir()

['split.py', 'test', 'train']

In [7]:
def lecturaDeClase(clase):
  imagenes = []
  nombre = os.path.basename(clase)
    
  #Recorrer todas las imagenes
  for d in os.listdir(clase):

    #Leer imagenes
    im = cv.imread(clase + '/' + d)
    im = cv.cvtColor(im,cv.COLOR_BGR2RGB)

    #Agregar a lista
    imagenes.append(im)

  print(f"{nombre}: {len(imagenes)} imágenes cargadas")

  return nombre, imagenes

In [9]:
def cargarClasesParalelizado(direc):
  #Obtener lista de carpetas de clases
  carpetas = [os.path.join(direc, nombre) for nombre in os.listdir(direc) if os.path.isdir(os.path.join(direc, nombre))]

  #Diccionario donde guardar {Clase: [imagenes]}
  imagenesClase = {}

  #Paralelizar carga de imágenes por cada clase
  with ThreadPoolExecutor() as executor:
    resultados = executor.map(lecturaDeClase, carpetas)

  #Juntar resultados
  for clase, imagenes in resultados:
    imagenesClase[clase] = imagenes

  return imagenesClase

In [11]:
def cargarClasesSecuencial(direc):
  #Obtener lista de carpetas de clases
  carpetas = [os.path.join(direc, nombre) for nombre in os.listdir(direc) if os.path.isdir(os.path.join(direc, nombre))]

  #Diccionario donde guardar {Clase: [imagenes]}
  imagenesClase = {}

  #Cargar imágenes de manera secuencial
  for d in carpetas:
    clase, imagenes = lecturaDeClase(d)
    imagenesClase[clase] = imagenes

  return imagenesClase

### Implementación

In [14]:
inicio_S = time.time()

imagenes_dict = cargarClasesSecuencial("train")

fin_S = time.time()
tiempo_total_S = fin_S - inicio_S

print(f"\nTiempo total de ejecución Secuencial: {tiempo_total_S:.4f} segundos")

bird: 1245 imágenes cargadas
cat: 1205 imágenes cargadas
dog: 1554 imágenes cargadas

Tiempo total de ejecución Secuencial: 108.8074 segundos


In [13]:
# Llamar la función
inicio_P = time.time()
imagenes_dict = cargarClasesParalelizado("train")

fin_P = time.time()
tiempo_total_P = fin_P - inicio_P

print(f"\nTiempo total de ejecución Paralelo: {tiempo_total_P:.4f} segundos")

bird: 1245 imágenes cargadas
cat: 1205 imágenes cargadas
dog: 1554 imágenes cargadas

Tiempo total de ejecución Paralelo: 50.6190 segundos


## 3) Extracción de Características

### Métodos

In [16]:
def vectorCaracteristicoSIFT(img):
  #Crear sift
  sift = cv.SIFT_create()

  #Obtener puntos claves y descriptores
  keypoints, descriptors = sift.detectAndCompute(img, None)

  #Vector Característico
  vec = np.mean(descriptors, axis = 0)

  return vec

In [17]:
def vectorCaracteristicoHOG(img):
    # Crear el descriptor HOG y utilizar el clasificador preentrenado para detección de personas
    hog = cv.HOGDescriptor()
    hog.setSVMDetector(cv.HOGDescriptor_getDefaultPeopleDetector())

    #Reescala las imagenes
    img_resized = cv.resize(img, (128,128))

    # Calcular el vector HOG
    vector_hog = hog.compute(img_resized)

    # Aplanar el vector HOG
    vector_hog = np.ravel(vector_hog)

    return vector_hog

In [18]:
def vectorCaracteristicoResultante(clase, tipo):
  VCR = []

  #Navegar por las imágenes de la clase
  for img in clase:
      #Extraccion de Vectores característicos de cada imagen
      if tipo == 'SIFT':
        VC = vectorCaracteristicoSIFT(img)
      elif tipo == 'HOG':
        VC = vectorCaracteristicoHOG(img)

      #Concatenar los vectores característicos
      VCR.append(VC)

  return VCR

In [19]:
def vectoresCaracteristicosParalelizado(diccionario, tipo):
  vCaracteristicoClase = {}

  with ThreadPoolExecutor() as executor:
  # Creamos una lista de tareas por clase
    futuras = {
      executor.submit(vectorCaracteristicoResultante, imagenes, tipo): clase
      for clase, imagenes in diccionario.items()
    }

    # Recolectamos resultados
    for futuro in futuras:
      clase = futuras[futuro]
      resultado = futuro.result()
      vCaracteristicoClase[clase] = resultado

  return vCaracteristicoClase

def vectoresCaracteristicosSecuencial(diccionario, tipo):
  vCaracteristicoClase = {}

  for clase, imagenes in diccionario.items():
    vCaracteristicoClase[clase] = vectorCaracteristicoResultante(imagenes, tipo)

  return vCaracteristicoClase

### Implementacion

In [23]:
inicio_S = time.time()

vCaracteristicos_dict_SIFT = vectoresCaracteristicosSecuencial(imagenes_dict, 'SIFT')
vCaracteristicos_dict_HOG = vectoresCaracteristicosSecuencial(imagenes_dict, 'HOG')

fin_S = time.time()
tiempo_total_S = fin_S - inicio_S

print(f"\nTiempo total de ejecución Secuencial: {tiempo_total_S:.4f} segundos")

KeyboardInterrupt: 

In [ ]:
#PARALELIZADO
inicio_P = time.time()

vCaracteristicos_dict_SIFT = vectoresCaracteristicosParalelizado(imagenes_dict, 'SIFT')
vCaracteristicos_dict_HOG = vectoresCaracteristicosParalelizado(imagenes_dict, 'HOG')

fin_P = time.time()
tiempo_total_P = fin_P - inicio_P

print(f"\nTiempo total de ejecución Paralelo: {tiempo_total_P:.4f} segundos")

## 4) Entrenamiento

In [97]:
# Convertimos a listas X (features) y y (labels)
X = []
y = []

for clase, vectores in vCaracteristicos_dict_SIFT.items():
    X.extend(vectores)
    y.extend([clase] * len(vectores))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
# Entrenamos el modelo
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

Accuracy: 0.24


In [101]:
svc = SVC(kernel='linear')  # O puedes usar 'rbf' si prefieres kernel radial
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

Accuracy: 0.32


In [107]:
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
gbm.fit(X_train, y_train)

y_pred = gbm.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

Accuracy: 0.33


In [108]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

Accuracy: 0.33


## 5) Pruebas

In [187]:
im = cv.imread("test\\Convertible\\11_jpg.rf.ea923c3a0ff80e933d12bfb74b350e06.jpg")
im = cv.cvtColor(im,cv.COLOR_BGR2RGB)

vector = vectorCaracteristicoSIFT(im)
prediccion = svc.predict([vector])
print(f'Predicción: {prediccion[0]}')

Predicción: train\Pick-Up


## 6) Guardar Modelos

In [185]:
joblib.dump(knn, 'Modelos/knn_model.pkl')
joblib.dump(svc, 'Modelos/svc_model.pkl')
joblib.dump(gbm, 'Modelos/gbm_model.pkl')
joblib.dump(rfc, 'Modelos/random_forest_model.pkl')

print("Guardados")

Guardados


In [189]:
knn_loaded = joblib.load('../Modelos/knn_model.pkl')

im = cv.imread("test\\Convertible\\11_jpg.rf.ea923c3a0ff80e933d12bfb74b350e06.jpg")
im = cv.cvtColor(im,cv.COLOR_BGR2RGB)

vector = vectorCaracteristicoSIFT(im)

y_pred = knn_loaded.predict(X_test)
print(f'Predicción: {prediccion[0]}')

Predicción: train\Pick-Up
